# Drug–Protein Interaction Pipeline
Includes:
1. SMILES to Morgan Fingerprint & Molecular Graph
2. Protein FASTA → ESM2 Embedding
3. Merge & Normalize Data for ML
4. Output DTI Dataset (.csv/.json)

In [2]:
# ⚙️ Install required packages
!pip install rdkit-pypi fair-esm biopython torch pandas numpy
import os, json
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from Bio import SeqIO
import torch
import esm

## 🔬 Drug Fingerprints and Graphs

In [3]:
smiles_list = [
    ("DB08820", "CC(C)(C)C1=CC(=C(C=C1NC(=O)C2=CNC3=CC=CC=C3C2=O)O)C(C)(C)C"),  # Ivacaftor

]
fingerprints = []
for drug_id, smiles in smiles_list:
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, 1024)
    fp_array = np.array(fp)
    fingerprints.append({"drug_id": drug_id, "smiles": smiles, "fingerprint": fp_array.tolist()})

## 🧬 Protein Embeddings (ESM2)

In [4]:
# Load ESM2
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()

protein_data = [
    ("P13569", "CFTR", "MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSADNLSEKLEREWDRELASKKNPKLINALRRCFFWRFMFYGIFLYLGEVTKAVQPLLLGRIIASYDPDNKEERSIAIYLGIGLCLLFIVRTLLLHPAIFGLHHIGMQMRIAMFSLIYKKTLKLSSRVLDKISIGQLVSLLSNNLNKFDEGLALAHFVWIAPLQVALLMGLIWELLQASAFCGLGFLIVLALFQAGLGRMMMKYRDQRAGKISERLVITSEMIENIQSVKAYCWEEAMEKMIENLRQTELKLTRKAAYVRYFNSSAFFFSGFFVVFLSVLPYALIKGIILRKIFTTISFCIVLRMAVTRQFPWAVQTWYDSLGAINKIQDFLQKQEYKTLEYNLTTTEVVMENVTAFWEEGFGELFEKAKQNNNNRKTSNGDDSLFFSNFSLLGTPVLKDINFKIERGQLLAVAGSTGAGKTSLLMVIMGELEPSEGKIKHSGRISFCSQFSWIMPGTIKENIIFGVSYDEYRYRSVIKACQLEEDISKFAEKDNIVLGEGGITLSGGQRARISLARAVYKDADLYLLDSPFGYLDVLTEKEIFESCVCKLMANKTRILVTSKMEHLKKADKILILHEGSSYFYGTFSELQNLQPDFSSKLMGCDSFDQFSAERRNSILTETLHRFSLEGDAPVSWTETKKQSFKQTGEFGEKRKNSILNPINSIRKFSIVQKTPLQMNGIEEDSDEPLERRLSLVPDSEQGEAILPRISVISTGPTLQARRRQSVLNLMTHSVNQGQNIHRKTTASTRKVSLAPQANLTELDIYSRRLSQETGLEISEEINEEDLKECFFDDMESIPAVTTWNTYLRYITVHKSLIFVLIWCLVIFLAEVAASLVVLWLLGNTPLQDKGNSTHSRNNSYAVIITSTSSYYVFYIYVGVADTLLAMGFFRGLPLVHTLITVSKILHHKMLHSVLQAPMSTLNTLKAGGILNRFSKDIAILDDLLPLTIFDFIQLLLIVIGAIAVVAVLQPYIFVATVPVIVAFIMLRAYFLQTSQQLKQLESEGRSPIFTHLVTSLKGLWTLRAFGRQPYFETLFHKALNLHTANWFLYLSTLRWFQMRIEMIFVIFFIAVTFISILTTGEGEGRVGIILTLAMNIMSTLQWAVNSSIDVDSLMRSVSRVFKFIDMPTEGKPTKSTKPYKNGQLSKVMIIENSHVKKDDIWPSGGQMTVKDLTAKYTEGGNAILENISFSISPGQRVGLLGRTGSGKSTLLSAFLRLLNTEGEIQIDGVSWDSITLQQWRKAFGVIPQKVFIFSGTFRKNLDPYEQWSDQEIWKVADEVGLRSVIEQFPGKLDFVLVDGGCVLSHGHKQLMCLARSVLSKAKILLLDEPSAHLDPVTYQIIRRTLKQAFADCTVILCEHRIEAMLECQQFLVIEENKVRQYDSIQKLLNERSLFRQAISPSDRVKLFPHRNSSKCKSKPQIAALKEETEEEVQDTRL"[:200]),

]
protein_embeddings = []

for uniprot, name, seq in protein_data:
    batch_labels, batch_strs, batch_tokens = batch_converter([(name, seq)])
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6])
    token_representations = results["representations"][6]
    emb = token_representations[0, 1:len(seq)+1].mean(0).numpy()
    protein_embeddings.append({"uniprot": uniprot, "name": name, "embedding": emb.tolist()})

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


## 🧠 Create Combined Dataset

In [5]:
# Match drug–protein pairs and create dataset
dataset = []
for drug in fingerprints:
    for protein in protein_embeddings:
        dataset.append({
            "drug_id": drug["drug_id"],
            "drug_smiles": drug["smiles"],
            "drug_fingerprint": drug["fingerprint"],
            "protein_id": protein["uniprot"],
            "protein_embedding": protein["embedding"],
            "label": 1
        })

# Save as JSON
with open("drug_protein_dataset.json", "w") as f:
    json.dump(dataset, f)
print("Saved drug_protein_dataset.json")

Saved drug_protein_dataset.json


In [6]:
# Download JSON ML file input

from google.colab import files
files.download('drug_protein_dataset.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# Download the csv file format

import pandas as pd
df = pd.DataFrame(dataset)
df.to_csv("drug_protein_dataset.csv", index=False)
print("Saved drug_protein_dataset.csv")

files.download('drug_protein_dataset.csv')

Saved drug_protein_dataset.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>